In [126]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
odds_filter=1.75
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
time_now_formatted = time_now.strftime("%Y-%m-%d")
data=pd.read_sql_query("Select distinct * from results_clay_1 where date > '2024-01-01'",con=devengine)
data["Fav_Odds"] = data.Fav_Odds.astype(float)
data["Dog_Odds"] = data.Dog_Odds.astype(float)
data['Fav_Win'] = data['Winner']==data['Fav']
data=data[data['Fav_Odds']>odds_filter]

In [122]:
test_data = data.copy()
test_data1 = data.copy()
test_data['Winner_Dog'] = test_data['Dog'] == test_data['Winner']
test_data['Winner_EloDog'] = test_data['Elo_Fav'] != test_data['Winner']
column_order = ['Winner_Dog', 'Winner_EloDog'] + [col for col in test_data if col not in ['Winner_Dog', 'Winner_EloDog']]
test_data = test_data[column_order]
test_data=test_data.drop(columns=['Winner_Odds'])
test_data=test_data[[ 'fav_percent', 'dog_percent','fav_rank', 'dog_rank', 'Elo_Fav_Elo', 'Elo_Dog_Elo', 'Fav_Top100', 'Dog_Top100','Winner','Fav','Dog','Fav_Odds','Dog_Odds','Elo_Fav']]

# Create a single set of random indices for all columns
random_order = np.random.permutation(test_data.index)

# Create new columns with randomized order for specific columns
test_data['Player_1'] = test_data.apply(lambda row: np.random.choice([row['Fav'], row['Dog']]), axis=1)
test_data['Player_2'] = test_data.apply(lambda x: x['Fav'] if x['Player_1']==x['Dog'] else x['Dog'],axis=1)
test_data['Player_1_Odds'] = test_data.apply(lambda x: x['Fav_Odds'] if x['Player_1']!=x['Dog'] else x['Dog_Odds'],axis=1)
test_data['Player_2_Odds'] = test_data.apply(lambda x: x['Fav_Odds'] if x['Player_1']==x['Dog'] else x['Dog_Odds'],axis=1)
test_data['Player_1_Rank'] = test_data.apply(lambda x: x['fav_rank'] if x['Player_1']!=x['Dog'] else x['dog_rank'],axis=1)
test_data['Player_2_Rank'] = test_data.apply(lambda x: x['fav_rank'] if x['Player_1']==x['Dog'] else x['dog_rank'],axis=1)
test_data['Player_1_Top100'] = test_data.apply(lambda x: x['Fav_Top100'] if x['Player_1']!=x['Dog'] else x['Dog_Top100'],axis=1)
test_data['Player_2_Top100'] = test_data.apply(lambda x: x['Fav_Top100'] if x['Player_1']==x['Dog'] else x['Dog_Top100'],axis=1)
test_data['Player_1_percent'] = test_data.apply(lambda x: x['fav_percent'] if x['Player_1']!=x['Dog'] else x['dog_percent'],axis=1)
test_data['Player_2_percent'] = test_data.apply(lambda x: x['fav_percent'] if x['Player_1']==x['Dog'] else x['dog_percent'],axis=1)
test_data['FavEloFav']=test_data['Fav']==test_data['Elo_Fav']
test_data['Fav_Elo']=test_data.apply(lambda x: x['Elo_Fav_Elo'] if (x['FavEloFav']==True) else x['Elo_Dog_Elo'],axis=1)
test_data['Dog_Elo']=test_data.apply(lambda x: x['Elo_Fav_Elo'] if (x['FavEloFav']==False) else x['Elo_Dog_Elo'],axis=1)

test_data['Player_1_Elo'] = test_data.apply(lambda x: x['Fav_Elo'] if x['Player_1']!=x['Dog'] else x['Dog_Elo'],axis=1)
test_data['Player_2_Elo'] = test_data.apply(lambda x: x['Fav_Elo'] if x['Player_1']==x['Dog'] else x['Dog_Elo'],axis=1)
test_data['Player_1_Win'] = test_data['Player_1']==test_data['Winner']
#test_data['Player_2_Elo'] = test_data.apply(lambda x: x['Elo_Dog_Elo'] if ((x['Player_1']==x['Dog'])&(x['FavEloFav2']==True)) else x['Elo_Fav_Elo'],axis=1)
# Filter columns with the word 'Player'
player_columns = [col for col in test_data.columns if 'Player' in col]

# Keep only the columns with the word 'Player'
test_data = test_data[player_columns]
test_data=test_data.drop(columns=['Player_1','Player_2','Player_1_Odds','Player_2_Odds'])

In [123]:
import numpy as np

# Assuming your DataFrame is named 'test_data'
data['fav_percent_adj'] = np.ceil(data['fav_percent'] * 10) / 10
data['dog_percent_adj'] = np.ceil(data['dog_percent'] * 10) / 10



In [124]:
# Assuming your DataFrame is named 'test_data'
result = data.groupby(['fav_percent_adj', 'dog_percent_adj'])['Fav_Win'].agg(['mean', 'count']).reset_index()

result.rename(columns={'mean': 'Fav_Win_Percentage', 'count': 'Match_Count'}, inplace=True)


result.sort_values(by='Fav_Win_Percentage').to_pickle('Fav_Win_Percentage_Clay')

In [132]:
hard_today=data
Fav_Win_Percentage=result
hard_today['fav_percent_adj'] = np.ceil(hard_today['fav_percent'] * 10) / 10
hard_today['dog_percent_adj'] = np.ceil(hard_today['dog_percent'] * 10) / 10
hard_today_win_percent=pd.merge(hard_today,Fav_Win_Percentage,left_on=['fav_percent_adj','dog_percent_adj'],right_on=['fav_percent_adj','dog_percent_adj'])
#hard_today_win_percent=hard_today_win_percent[hard_today_win_percent['Resulted']!='Falxse']
hard_today_win_percent=hard_today_win_percent[hard_today_win_percent['Sex']!='Womens']
results=hard_today_win_percent[['Fav_Win_Percentage','Fav','Fav_Odds','fav_percent','dog_percent','Dog','Dog_Odds','Fav_Win']]#.sort_values(by=['Time'],ascending=True )

results=results[results['Fav_Win_Percentage']<.4]
len(results[results['Fav_Win']==True])/len(results),len(results)

(0.42857142857142855, 7)